In [1]:
import json
from datetime import datetime

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

from model.sequential_classfier import SequentialClassifier
from type.abbreviation import Abbreviation
from type.features import CrfFeatures, CrfLabelSequence
from pydantic import BaseModel

In [2]:
class FeaturesScore(BaseModel):
    features: list[str]
    score: float

features_score = list(map(FeaturesScore.model_validate, json.load(open('features.json'))))

In [3]:
CrfFeatures.no_use_keys = [
]

In [4]:
# 前処理した略語データを読み込む
data = list(map(Abbreviation.model_validate, json.load(open("./data/abbreviation.json", "r"))))
X = [*map(CrfFeatures.from_abbreviation, data)]
y = list(map(CrfLabelSequence.from_abbreviation, data))
# ラベルの番号振り
CrfFeatures.assign_feature_idx(X)
print()

In [5]:
model = SequentialClassifier(
    RandomForestClassifier(
        random_state=616,
        n_estimators=393,
        max_depth=19,
        max_features="sqrt",
        criterion="entropy",
    )
)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=616)

In [7]:
scores: list[float] = []
for train_idx, test_idx in kf.split(X):
    X_train = [X[i] for i in train_idx]
    y_train = [y[i] for i in train_idx]
    X_test = [X[i] for i in test_idx]
    y_test = [y[i] for i in test_idx]
    model.fit(X_train, y_train)
    case_score_list = model.score(X_test, y_test)
    score = np.mean(case_score_list)
    scores.append(score)

score_mean = np.mean(scores)

KeyboardInterrupt: 

In [8]:
print(score_mean)

fs = FeaturesScore(
    features=[f for f in CrfFeatures.model_json_schema()['required'] if f not in CrfFeatures.no_use_keys],
    score=score_mean,
)

features_score.append(fs)

NameError: name 'score_mean' is not defined

In [ ]:
# json.dump([fs.model_dump() for fs in features_score], open('features.json', 'w'), ensure_ascii=False, indent=4)